In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import preprocessing
from sklearn import tree
from typing import List
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [45]:

import random
import math
def generateData(n):
    data = []
    for i in range(0,n):
        minggu = random.randint(1, 392)
        totaldonor = 1 if math.floor(minggu/8) == 0 else random.randint(1,math.floor(minggu/8))
        data.append([minggu, totaldonor,0.0 if totaldonor == 0 else 1.0 if minggu/totaldonor <= 12.0 else 0.0])
    return data

data = generateData(2000)

real_df = pd.DataFrame(data,columns=["Jumlah Minggu", "Total donor per tahun","aktif"])
real_df.head()

,Jumlah Minggu,Total donor per tahun,aktif
0,72,4,0.0
1,3,1,1.0
2,114,7,0.0
3,13,1,0.0
4,151,2,0.0


In [46]:
def rules(n):
    totalminggu = n[0]
    totaldonor = n[1]
    totalminggu_now = n[2]
    totaldonor_now = n[3]
    if totaldonor <= 1:
        return 0.0
    
    if totalminggu/totaldonor < 12:
        return 1.0
    
    if totaldonor_now <= 1:
        return 0.0
    # if totaldonor_now == 0:

    if totalminggu_now/totaldonor_now < 12:
        return 1.0
    
    return 0.0


In [47]:

import random
import math


def generateData(n, with_label=True):
    data = []
    for i in range(0, n):
        minggu = random.randint(1, 392)

        totaldonor = 1 if math.floor(
            minggu/8) == 0 else random.randint(1, math.floor(minggu/8))
        totalminggu_now = random.randint(0, 52)
        totaldonor_now = random.randint(0, int(min(totaldonor-1,totalminggu_now/8)))
        if totalminggu_now == 0 or totaldonor_now == 0:
            totalminggu_now = 0
            totaldonor_now = 0
        d = [minggu, totaldonor, totalminggu_now, totaldonor_now]
        if with_label:
            data.append(d + [rules(d)])
        else:
            data.append(d)
    return data


data = generateData(2000)

real_df = pd.DataFrame(data, columns=["Jumlah Minggu Kesuluruhan", "Total Donor Keseluruhan",
                       "Jumlah Minggu Tahun Ini", "Total Donor Tahun Ini", "aktif"])
real_df.tail()


,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Tahun Ini,Total Donor Tahun Ini,aktif
995,112,6,40,2,0.0
996,269,24,52,4,1.0
997,12,1,0,0,0.0
998,25,1,0,0,0.0
999,149,10,0,0,0.0


In [48]:
scalerX = MinMaxScaler()


def preprocessing(df:pd.DataFrame,categorical_columns:List[str],scaling:bool, unused_columns:List[str], label:str):
  df.dropna() # drop row bila salah 1 atau lebih ada NaN
  # df.dropna(subset=[1]) # drop row bila column 1 NaN

  if len(categorical_columns) > 0:
    df = pd.get_dummies(data=df,columns=categorical_columns)
  # for column in df.columns:
  #   le = LabelEncoder()
  #   le.fit(df[column])
  #   df[column] = le.transform(df[column])
  
  # harus displit dulu antara features dan label, karena bila di minmax scalling dulu
  # nama kolom akan hilang, berubah menjadi array 2d
  unused_columns.append(label) # hapus kolom data column dan juga label
  X = df.drop(columns=unused_columns) 
  Y = df[label]
  Y = np.array(Y).reshape((-1,1))
  if scaling:
    global scalerX
    scalerX.fit(X.to_numpy())
    print("X")
    print(X)
    X = scalerX.transform(X.to_numpy())

    scaler = MinMaxScaler()
    scaler.fit(Y)
    Y = scaler.transform(Y)

  # always produce the same result (only random once)
  # x_train, x_test = train_test_split(X, test_size=test_size, random_state=25)
  # y_train, y_test = train_test_split(Y, test_size=test_size, random_state=25)

  # x_train = np.array(x_train)
  # y_train = np.array(y_train).reshape((-1,1))
  # x_test  = np.array(x_test)
  # y_test  = np.array(y_test).reshape((-1,1))
  X = np.array(X)
  Y = np.array(Y).reshape((-1,1))
  
  return X, Y


In [49]:
real_df.head()

,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Tahun Ini,Total Donor Tahun Ini,aktif
0,239,20,31,2,1.0
1,328,36,0,0,1.0
2,336,11,0,0,0.0
3,207,3,13,1,0.0
4,172,17,43,4,1.0


In [50]:
x_train, y_train = preprocessing(real_df,
                                                 scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="aktif")

In [51]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [52]:
scaled_x_train, scaled_y_train = preprocessing(real_df,
                                                 scaling=True,
                                                 categorical_columns=[],
                                                 unused_columns=[],
                                                 label="aktif")

X
     Jumlah Minggu Kesuluruhan  Total Donor Keseluruhan  \
0                          239                       20   
1                          328                       36   
2                          336                       11   
3                          207                        3   
4                          172                       17   
..                         ...                      ...   
995                        112                        6   
996                        269                       24   
997                         12                        1   
998                         25                        1   
999                        149                       10   

     Jumlah Minggu Tahun Ini  Total Donor Tahun Ini  
0                         31                      2  
1                          0                      0  
2                          0                      0  
3                         13                      1  
4                  

In [53]:

# tranfusion_arr = tranfusion_df.to_numpy()
real_case_arr = generateData(100,False)

# tranfusion_arr =np.array(list(map(toWeek,tranfusion_arr)))
scaled_real_case_arr = scalerX.transform(real_case_arr)

In [54]:
scaled_y_train

array([[1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],

In [55]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation="relu",solver="lbfgs",shuffle=True)
mlp.fit(scaled_x_train, scaled_y_train.ravel())

MLPClassifier(solver='lbfgs')

In [56]:
real_case_arr[0:5]

[[146, 2, 0, 0],
 [157, 3, 11, 1],
 [64, 4, 0, 0],
 [101, 7, 0, 0],
 [385, 10, 51, 6]]

In [57]:
expectValue = list(map(lambda x: rules(x), real_case_arr))
expectValue[0:5]

[0.0, 0.0, 0.0, 0.0, 1.0]

expectValue digunakan untuk menvalidasi output dari model MLP dan Decision Tree  
Berdasarkan https://www.halodoc.com/artikel/berapa-kali-boleh-donor-darah-dalam-setahun dikatakan aktif(pria) bila interval waktu donor 8-14 minggu

 **source dataset  : https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download**

In [58]:
DTresult = model.predict(real_case_arr) 
# MLPresult = mlp.predict(scaled_real_case_arr)

real_case_df = pd.DataFrame(real_case_arr,columns=["Jumlah Minggu Kesuluruhan", "Total Donor Keseluruhan", "Jumlah Minggu Tahun Ini", "Total Donor Tahun Ini"])
# real_case_df["Predicted Value (MLP)"] = MLPresult
real_case_df["Predicted Value (DT)"] = DTresult
real_case_df["Expected Value"] = expectValue

In [59]:
from IPython.display import display, HTML
# tranfusion_df
display(HTML("<h2>Hasil  Decision Tree (DT)</h2><p>source dataset: <a href='https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download'>kaggle</a></p>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{real_case_df.to_html()}</div>"))

,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Tahun Ini,Total Donor Tahun Ini,Predicted Value (DT),Expected Value
0,146,2,0,0,0.0,0.0
1,157,3,11,1,0.0,0.0
2,64,4,0,0,0.0,0.0
3,101,7,0,0,0.0,0.0
4,385,10,51,6,1.0,1.0
5,145,17,26,1,1.0,1.0
6,365,15,0,0,0.0,0.0
7,321,5,0,0,0.0,0.0
8,51,1,0,0,0.0,0.0
9,1,1,0,0,0.0,0.0


In [60]:
# Modified input
def modified(data,no_label=False):
    modified_data = []
    for i, d in enumerate(data):
        if d[1] == 0:
            interval_all = 0
        else:
            interval_all = d[0]/d[1]

        if d[3] == 0:
            interval_now = 0
        else:
            interval_now = d[2]/d[3]

        if no_label:
            modified_data.append([interval_all,interval_now,d[1],d[3]]) # kedua input digabung
        else:
            modified_data.append([interval_all,interval_now,d[1],d[3],d[4]]) # kedua input digabung
    return modified_data
modified_data = modified(data)
modified_data
modified_df = pd.DataFrame(modified_data,columns=["Mean Interval keseluruhan","Mean Interval terbaru","Total Donor keseluruhan","Total Donor terbaru","aktif"])
modified_df.head()

,Mean Interval keseluruhan,Mean Interval terbaru,Total Donor keseluruhan,Total Donor terbaru,aktif
0,11.950000,15.50,20,2,1.0
1,9.111111,0.00,36,0,1.0
2,30.545455,0.00,11,0,0.0
3,69.000000,13.00,3,1,0.0
4,10.117647,10.75,17,4,1.0


In [61]:
modifiedDT = tree.DecisionTreeClassifier()
modified_x_train, modified_y_train = preprocessing(modified_df, scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="aktif")
modifiedDT.fit(modified_x_train, modified_y_train)
real_case_arr_modified = modified(real_case_arr,no_label=True)
modifiedDTresult = modifiedDT.predict(real_case_arr_modified) 

In [62]:
real_case_df["Predicted Value (modified DT)"] = modifiedDTresult

display(HTML("<h2>Hasil Perbandingan Deep Learning (MLP) dan Decision Tree (DT) dan Modified Decision Tree (DT)</h2><p>source dataset: <a href='https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download'>kaggle</a></p>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{real_case_df.to_html()}</div>"))

,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Tahun Ini,Total Donor Tahun Ini,Predicted Value (DT),Expected Value,Predicted Value (modified DT)
0,146,2,0,0,0.0,0.0,0.0
1,157,3,11,1,0.0,0.0,0.0
2,64,4,0,0,0.0,0.0,0.0
3,101,7,0,0,0.0,0.0,0.0
4,385,10,51,6,1.0,1.0,1.0
5,145,17,26,1,1.0,1.0,1.0
6,365,15,0,0,0.0,0.0,0.0
7,321,5,0,0,0.0,0.0,0.0
8,51,1,0,0,0.0,0.0,0.0
9,1,1,0,0,0.0,0.0,0.0


In [63]:
# fig = plt.figure(figsize=(25,20))
# _ = tree.plot_tree(model, 
#                    feature_names=["Donor 1 tahun terakhir","Jumlah hari terakhir donor", "frekuensi donor per tahun"],  
#                    class_names=["aktif"],
#                    filled=True)

import graphviz
# DOT data
dot_data = tree.export_graphviz(modifiedDT, out_file=None, 
                                feature_names=["interval donor seluruhnya","interval donor sekarang","Total donor seluruhnya","Total donor now"],  
                                class_names=["Aktif","non Aktif"],
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph.render("decision_tree_graphivz")

'decision_tree_graphivz.png'

In [64]:
import pickle
pickle.dump(modifiedDT, open("model", 'wb'))